# NLP : Assignment 3
### Rohit Roy | MDS202340

In [1]:
from collections import Counter, defaultdict
from wiki_dump_reader import Cleaner , iterate

 
from nltk import ngrams, word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from scipy import sparse
from scipy.sparse import dok_matrix
from scipy.spatial.distance import cosine
from scipy.sparse.linalg import svds

import numpy as np
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from nltk.tokenize import word_tokenize

corpus_file = 'HindiCorpus.txt'
preprocessed_file='ProcessedCorpus.txt' 


MInimize corpus

In [2]:
def minimize_pre(so,de):
	with open(so, 'r',encoding="utf-8") as sor:
		with open(de, 'w',encoding="utf-8") as des:
				for line in sor:
					des.write(line[:(len(line)//4)])
	
minimize_pre('ProcessedCorpus-Org.txt','ProcessedCorpus.txt')

#### 1. Building Vocabulary

I am using the Bengali corpus from the 1st assignment, which was split into 8 files each of 100000 lines.

In [1]:
def write_corpus ():
	
	page_count = 0
	cleaner = Cleaner ()
	with open(corpus_file , 'w', encoding='utf -8') as output:
		for title , text in iterate('hiwiki-latest-pages-articles.xml'):
			text = cleaner.clean_text(text)
			cleaned_text , _ = cleaner.build_links(text)
			output.write(title + '\n' + cleaned_text + '\n ')
			page_count += 1
			if page_count % 1000 == 0:
				print(f'Pages dumped = {page_count}', end='\r')
				#print(title + '\n' + cleaned_text + '\n')
		output.close ()
	print(f"\npage count = {page_count}")


write_corpus ()

NameError: name 'Cleaner' is not defined

In [105]:

import string

stop_words=[]

start = int('0x0900', 16)
end = int('0x097F', 16)

num_start = int('0x0966', 16)
num_end = int('0x096F', 16)

hindi_letters = [chr(code) for code in range(start, end + 1)]
hindi_letters.remove('॥')
hindi_letters.remove('।')
hindi_letters.remove('ॽ')

hindi_numbers = [chr(code) for code in range(num_start, num_end + 1)]

def create_stop_words(file_path):
	try :
		with open(file_path, 'r',encoding="utf-8") as file:
			for line in file:
				stop_words.append(line.strip())
	except:
		print()

def remove_numbers(text):
	tokens = list(text)
	filtered_tokens = []

	for token in tokens:
		if token not in hindi_numbers:
			filtered_tokens.append(token)

	result = ''.join(filtered_tokens)
	return result

def remove_punctuation(text):
	translator = str.maketrans('', '', string.punctuation)
	return text.translate(translator)

def remove_whitespace(text):
	return " ".join(text.split())
			
file_path = 'stopwords.txt'  # Replace 'example.txt' with the path to your file
create_stop_words(file_path)


def preprocess_line(text):
	text=remove_whitespace(remove_punctuation(remove_numbers(text)))
	
	words = text.split(" ")

	# Filter out foreign words
	filtered_tokens = []
	for word in words:
		if word in stop_words:
			continue
		
		breaker=False
		for letter in word:
			if letter not in hindi_letters:
				breaker=True
				break
		
		if breaker:
			continue
			
			
		filtered_tokens.append(word)
	
	# Reassemble the text
	processed_text = ' '.join(filtered_tokens)
	
	return processed_text


def preprocess_corpus(source_file, destination_file):
	page_count=0
	with open(source_file, 'r',encoding="utf-8") as source:
		with open(destination_file, 'w',encoding="utf-8") as destination:
			for line in source:
				destination.write(preprocess_line(line))
				page_count+=1
				if page_count % 1000 == 0:
					print(f'Lines processed = {page_count}', end='\r')

# Example usage
preprocessed_file='ProcessedCorpus.txt' 
source_file_path = corpus_file  # Replace 'source.txt' with the path to your source file
destination_file_path = preprocessed_file  # Replace 'destination.txt' with the path to your destination file

preprocess_corpus(source_file_path, destination_file_path)

In [3]:

def gen_token_count(file):
	token_len=0
	with open(file, 'r',encoding="utf-8") as f:
		for line in f:
			space_count=0
			for char in line:
				if char == ' ':
						space_count += 1

			token_len+=space_count+1
			
	return token_len

def gen_token(file):
	tokens=[]
	
	with open(file, 'r',encoding="utf-8") as f:
		for line in f:
			tokens+=(line.split(' '))
			
	return tokens

def gen_vocabulary(file):
	word_list=[]
	with open(file, 'r',encoding="utf-8") as f:
		for line in f:
			word_list+=(line.split(' '))
			
	word_counts = Counter(word_list)
	return word_counts

In [4]:
corpus_token_count=gen_token_count(corpus_file)
print("Size of Corpus Tokens :",corpus_token_count)

preprocess_token_count=gen_token_count(preprocessed_file)
print("Size of Pre Processed Corpus Tokens :",preprocess_token_count)



Size of Corpus Tokens : 86699267
Size of Pre Processed Corpus Tokens : 10575437


In [5]:
vocab_counts = gen_vocabulary(preprocessed_file)
print("Size of vocab :",len(vocab_counts))


Size of vocab : 863935


In [6]:

vocab = {x for x, count in vocab_counts.items() if count >= 100}
print("Size of reduced vocab :", len(vocab))


# Number to word
vocab_list = list(vocab)

# Corresponding word to number
vocab_pos = {vocab_list[i] : i for i in range(len(vocab_list))}

# Compiling both into a dictionary
vocab_idx = vocab_pos.copy()
vocab_idx.update({i : w for i,w in enumerate(vocab_list)})


Size of reduced vocab : 9270


#### 2. Building Co-occurance matrix

The function below returns a co-occurance matrix, which is a csr matrix.

In [7]:
def co_occurances(file):
	window = 5

	# Using a ramp of window 4
	ramp = [0] + [*range(window,0,-1)]     

	occurances = defaultdict(lambda : Counter())

	with open(file, 'r',encoding="utf-8") as corpus:
		for line in corpus:
			all_grams = ngrams(word_tokenize(line), window+1, pad_right = True, pad_left = True)
			for grams in all_grams:

				if grams[0] in vocab :
					for idx, gram in enumerate(grams):
						if gram in vocab:
							occurances[vocab_idx[grams[0]]][vocab_idx[gram]] += ramp[idx]

				# Doing the same as above with the gram reversed
				grams_rev = grams[::-1]

				if grams_rev[0] in vocab:
					for idx, gram in enumerate(grams_rev):
						if gram in vocab:
							occurances[vocab_idx[grams_rev[0]]][vocab_idx[gram]] += ramp[idx]

	# Initializing a 0 matrix
	mat = np.zeros((len(vocab), len(vocab)), dtype=np.int64)

	# Filling the matrix
	for i in tqdm(range(len(vocab))):
		for j in occurances[i].keys():
			mat[i][j] = occurances[i][j]

	return mat

In [8]:
cocrmat = co_occurances(preprocessed_file)

100%|██████████| 9270/9270 [00:01<00:00, 4803.41it/s]


In [9]:

def calculate_probability(co_occurrence_matrix):
	# Calculate row sums and column sums
	row_sums = np.sum(co_occurrence_matrix, axis=1)
	col_sums = np.sum(co_occurrence_matrix, axis=0)
	
	# Total co-occurrences
	N = np.sum(co_occurrence_matrix)
	
	# Calculate pi and pj
	pi = row_sums / N
	pj = col_sums / N
	
	return pi, pj
# Calculate pi and pj
pi, pj = calculate_probability(cocrmat)


In [28]:

def gen_ppmi_matrix(co_occurrence_matrix):
	ppmi_matrix=np.zeros (( len(vocab) , len(vocab)))

	row_sums = np.sum(co_occurrence_matrix, axis=1)
	for	i in tqdm(range(len(vocab))):
		for j in range(len(vocab)):

			if(row_sums[i]==0 or co_occurrence_matrix[i][j]==0):
				ppmi_matrix[i][j]=0
				continue
			pij=((co_occurrence_matrix[i][j]))/row_sums[i]
			ppmi_matrix[i][j]=max(0,math.log2(pij/(pi[i]*pj[j])))

	return ppmi_matrix

ppmi_matrix=gen_ppmi_matrix(cocrmat)

100%|██████████| 9270/9270 [00:34<00:00, 265.65it/s]


In [29]:
print((ppmi_matrix[0][0]))
print(type(ppmi_matrix[0][0]))

23.301875711325586
<class 'numpy.float64'>


In [17]:
count =0
for i in tqdm(range(len(ppmi_matrix))):
	for j in range(len(ppmi_matrix)):
		if(ppmi_matrix[i][j]==math.inf):
			count+=1
print(count)

100%|██████████| 9270/9270 [01:29<00:00, 103.66it/s]

0


#### 3. Building Word-embedding matrix

We iterate through the non-zero entries of the co-occurance matrix and calculate the correlation as per the formula given in the COALS paper. We don't care about the 0 entries since it is going to give us negative correlation anyway.

In [59]:

top_15 = vocab_counts.most_common(15)
top_15=[i for i,_ in top_15]
print("Top 15 popular words : ", top_15)

Top 15 popular words :  ['के', 'राज्य', 'रूप', 'नहीं', 'प्रदेश', 'नाम', 'घी', 'भारत', 'अधिक', 'क्षेत्र', 'स्थित', 'निर्माण', 'सबसे', 'जिले', 'दक्षिण']


#### 4. Finding Similar Words

We find the k closest words to the selected word using the closest_words function which also returns the cosine distance between them.

In [60]:
nouns = [top_15[1],top_15[2],top_15[4],top_15[5],top_15[6],top_15[7],top_15[9],top_15[11],top_15[13],top_15[14]]
print(nouns)

['राज्य', 'रूप', 'प्रदेश', 'नाम', 'घी', 'भारत', 'क्षेत्र', 'निर्माण', 'जिले', 'दक्षिण']


In [49]:
import numpy as np

def calc_norm(vec):
	sum_of_squares = sum(x**2 for x in vec)
	return math.sqrt(sum_of_squares)

def cosine_similarity(vec1, vec2):
	v1=list(vec1)
	v2=list(vec2)
	
	dot_product=(0.0)

	for i in range(len(v1)):
		temp=dot_product
		dot_product+=(v1[i]*v2[i])
	#print(dot_product)
	norm_vec1 = calc_norm(v1)
	norm_vec2 = calc_norm(v2)
	return dot_product / (norm_vec1 * norm_vec2)

def reshape_top(words, extra):
	low = 0
	high = len(words) - 1

	while low <= high:
		mid = (low + high) // 2
		if words[mid][1] < extra[1]:
			low = mid + 1
		elif words[mid][1] > extra[1]:
			high = mid - 1
		else:
			break

	words.insert(low, extra)
	return words[1:]

def find_similar_words(target_word, vocab, ppmi_matrix, top_n=10):
	top_n+=1
	if target_word not in vocab:
		print("Target word not found in the vocabulary.")
		return
	
	target_index = vocab_pos[target_word]
	target_vector = ppmi_matrix[target_index]

	similarities = [(vocab_idx[0],0.0) for _ in range(top_n)]

	for i, vector in enumerate(ppmi_matrix):
		sim = cosine_similarity(target_vector, vector)
		similarities=reshape_top(similarities,((vocab_idx[i], sim)))

	similarities.reverse()

	return similarities[1:]



In [61]:
for noun in nouns:
	target_word = noun
	similar_words = find_similar_words(target_word, vocab, ppmi_matrix)

	print(f"Words similar to '{target_word}':")
	for word, similarity in similar_words:
		print(f"	{word}: {similarity}")

KeyboardInterrupt: 